In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

# Scikit-Learn
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

# PyTorch
import torch
import torch.nn as nn

# PyTorch Lightning
import pytorch_lightning as pl

# From src
from src.inference import format_predictions, calculate_metrics
from src.models import LongShortTermMemory
from src.model_training import TrainingLoop

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device} is available.")

In [ ]:
# Seed
seed = 6715
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
pl.seed_everything(seed)

In [ ]:
# Config
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 256 if AVAIL_GPUS else 32
EPOCHS = 100 if AVAIL_GPUS else 20

In [ ]:
train = torch.load("../data/train.pt")
val = torch.load("../data/val.pt")
test = torch.load("../data/test.pt")

# Training the Network

In [ ]:
# Hyperparameters
input_size = next(iter(train))[0].shape[0]
output_size = next(iter(train))[1].shape[0]
hidden_size = 64
num_layers = 2
dropout_prob = 0.1
learning_rate = 1e-2

optimizer = 'Adam'
loss_fn = nn.MSELoss()

In [ ]:
# Build model
model = LongShortTermMemory(
    hidden_size = hidden_size,
    num_rnn_layers = num_layers,
    input_size = input_size,
    output_size = output_size,
    dropout = dropout_prob,
    device = device
    )


print(model)

In [ ]:
train_loop = TrainingLoop(
            model=model, 
            datasets=(train, val, test),
            learning_rate=learning_rate,
            batch_size=BATCH_SIZE,
            optimizer=optimizer,
            accelerator=device,
            train_shuffle=False,
            num_dl_workers=2
        )

In [ ]:
trainer = pl.Trainer(
    accelerator=device,
    devices=1,
    min_epochs=1,
    max_epochs=EPOCHS,
    gradient_clip_val=0,
    check_val_every_n_epoch=1,
    val_check_interval=1.0
)

In [ ]:
# Run training
trainer.fit(train_loop)

# Inference

In [ ]:
# Run predictions
trainer.test(ckpt_path='best')

In [ ]:
predictions, values = train_loop.predictions, train_loop.values
#predictions = [tensor.cpu() for tensor in predictions]
#values = [tensor.cpu() for tensor in values]

preds = np.concatenate(predictions, axis=0)
vals = np.concatenate(values, axis=0)

scaler = pickle.load(open('../data/scaler.pkl', 'rb'))

preds = scaler.inverse_transform(preds)
vals = scaler.inverse_transform(vals)

In [ ]:
target_cols = [f'load+{h}' for h in range(1, 25)]
test_index = pd.date_range("2021-09-11 14:00:00", "2022-05-14 23:00:00", freq='h')

df_preds = pd.DataFrame(preds, columns=target_cols, index=test_index[:len(vals)])
df_vals = pd.DataFrame(vals, columns=target_cols, index=test_index[:len(vals)])

In [ ]:
print('TEST PERFORMANCE')
print('RMSE:\t', mean_squared_error(df_vals, df_preds)**0.5)
print('MAE:\t', mean_absolute_error(df_vals, df_preds))
print('MAPE:\t',mean_absolute_percentage_error(df_vals, df_preds) * 100)
print('R^2:\t', r2_score(df_vals, df_preds))

In [ ]:
forecast_time = '2021-09-11 14:00:00'
plt.plot(df_preds.loc[forecast_time], label='Prediction')
plt.plot(df_vals.loc[forecast_time], label='True')
plt.legend()
plt.grid()
plt.title(f"Forecast at: {forecast_time}")
plt.xticks(rotation=90)
plt.show()